In [3]:
import os
import sys
from collections import defaultdict
import pickle
import numpy as np
from numpy import gradient
from numpy.linalg import norm
from math import pi
from packages import data_container
from packages.data_container import Data
from packages.helper import traj_speed, beta, min_dist, d_psi, min_sep
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

In [7]:
'''Import experiment 1 data'''

header=('subj_x', 'subj_y', 'subj_yaw')
info={'p_goal':[], 'p_obst':[], 'obst_angle':[], 'obst_speed':[], 'subj_id':[], 'trial_id':[], 'stimuli_onset':[], 
      'stimuli_out':[], 'w_goal': 0.1, 'w_obst': 0.1, 'order':[], 'ps_trial':[], 'ps_subj':[], 'time_stamp':[], 
      'pass_order':[], 'p_subj':[], 'v_subj':[], 'a_subj':[], 'v_obst':[]}
data = Data(Hz=30, header=header, info=info)

file_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data\\Cohen_movObst\\Cohen_movObst_exp1_rawData.csv'))

ii = 0
head = 0
tail = 0
# subj_ps = {}
ps = 0
with open(file_path, 'r') as f:
    trials = [line.split(',') for line in f.read().split('\n')]
head = 0
while True:
    for i in range(head+1, len(trials)):
        if trials[i][0] == '':
            tail = i - 1
            break
    i_subj = int(trials[head][1])
    i_trial = int(trials[head][3])
    time_stamp = [float(line[0])/data.Hz for line in trials[head+2:tail+1]]
    subj = np.array([[float(line[5])/100, float(line[6])/100, float(line[7])] for line in trials[head+2:tail+1]])
    goal = np.array([[float(line[1])/100, float(line[2])/100] for line in trials[head+2:tail+1]])
    obst = np.array([[float(line[3])/100, float(line[4])/100] for line in trials[head+2:tail+1]])
    obst = Data.filter(obst, time_stamp, data.Hz, 4, 0.6)
    onset = 0         
    for i in range(len(subj)):
        if subj[i][1] >= 1:
            onset = i
            break

    data.add_traj(subj)
    p0 = Data.filter(subj[:,:2], time_stamp, data.Hz, 4, 0.6)
    v0 = gradient(p0, axis=0) * data.Hz
    a0 = gradient(v0, axis=0) * data.Hz

    # Use average speed at between -2 to -1 second as the trial preferred speed
    if len(subj) > 2 * data.Hz:
        ps = np.mean(traj_speed(subj[-2*data.Hz:-data.Hz,0:2], data.Hz))
    else:
        ps = 0

    # After inspection, those trials are no good
    if len(obst) < 32 or subj[-1][1] < 0:
        data.add_dump(ii, 'recording error')

    if norm(obst[-1,:] - obst[-31,:]) < 0.42:
        s_obst = 0.4
    elif norm(obst[-1,:] - obst[-31,:]) < 0.62:
        s_obst = 0.6
    elif norm(obst[-1,:] - obst[-31,:]) < 0.82:
        s_obst = 0.8
    else:
        s_obst = None

    if obst[0,1] < 4.1:
        angle = 70
    elif obst[0,1] < 4.8:
        angle = 90
    elif obst[0,1] < 5.4:
        angle = 110
    else:
        angle = None

    if obst[0,0] > 0 and angle:
        angle = -angle
        
    # Compute pass order
    for i in range(onset, len(subj) - 1):
        _beta = beta(p0[i], obst[i], v0[i])
        if abs(_beta) > pi / 2:
            stimuli_out = i
            # 1: pass in front of obst, -1: pass behind obst, 0: freewalk trial
            pass_order = np.sign(_beta * -angle)
            break
    v1 = (obst[-1] - obst[onset]) / (len(obst) - 1 - onset) * data.Hz        
    data.add_info({'p_goal':goal, 'p_obst':obst, 'obst_angle':angle, 'obst_speed':s_obst, 'subj_id':i_subj, 
                   'trial_id':i_trial, 'stimuli_onset':onset, 'stimuli_out':stimuli_out, 'ps_trial':ps,
                   'ps_subj':0, 'time_stamp':time_stamp, 'pass_order':pass_order,
                   'p_subj':p0, 'v_subj':v0, 'a_subj':a0, 'v_obst':v1})
    if tail + 3 >= len(trials):
        break
    head = tail + 2
    ii += 1
        
# Setting subject preferred speed to every trial
ps_subj = defaultdict(list)
for i in range(len(data.trajs)):
    subj_id = data.info['subj_id'][i]
    ps_subj[subj_id].append(data.info['ps_trial'][i])
for i in range(len(data.trajs)):
    subj_id = data.info['subj_id'][i]
    data.info['ps_subj'][i] = np.mean(ps_subj[subj_id])
    
outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data\\Cohen_movObst_exp1_data.pickle'))        
with open(outfile, 'wb') as file:   
    pickle.dump(data, file, pickle.HIGHEST_PROTOCOL)       

In [16]:
'''Check imported data'''
%matplotlib qt
vals = []
for i in range(len(data.trajs)):
    if i not in data.dump:
        onset = data.info['stimuli_onset'][i]
        p_subj = np.array(data.trajs[i])
        p_goal = np.array(data.info['p_goal'][i])
        p_obst = np.array(data.info['p_obst'][i])
        angle = data.info['obst_angle'][i]
        vals.append(data.info['ps_subj'][i])
        speed = traj_speed(np.array(data.get_traj(i))[:,0:2], data.Hz)
#         plt.plot(speed)
        plt.plot(vals)
#         if angle == 70 or angle == -70:
#             plt.plot(p_obst[:,0], p_obst[:,1])


In [6]:
data.info['stimuli_out']

[157,
 151,
 155,
 154,
 159,
 152,
 151,
 150,
 149,
 152,
 150,
 156,
 146,
 159,
 173,
 153,
 144,
 150,
 149,
 145,
 148,
 153,
 144,
 147,
 158,
 159,
 158,
 164,
 161,
 163,
 160,
 164,
 157,
 158,
 166,
 160,
 168,
 178,
 173,
 171,
 159,
 166,
 152,
 160,
 155,
 149,
 154,
 153,
 151,
 147,
 155,
 152,
 159,
 151,
 157,
 155,
 162,
 163,
 165,
 163,
 155,
 156,
 152,
 150,
 154,
 163,
 184,
 151,
 158,
 157,
 159,
 153,
 168,
 159,
 151,
 169,
 157,
 159,
 177,
 139,
 156,
 155,
 198,
 168,
 162,
 159,
 146,
 154,
 224,
 154,
 114,
 137,
 182,
 164,
 157,
 137,
 166,
 153,
 174,
 178,
 182,
 157,
 174,
 170,
 142,
 179,
 171,
 153,
 194,
 176,
 165,
 170,
 173,
 180,
 145,
 164,
 144,
 140,
 138,
 98,
 165,
 141,
 150,
 190,
 164,
 174,
 177,
 170,
 177,
 179,
 164,
 166,
 158,
 173,
 175,
 150,
 152,
 165,
 180,
 160,
 166,
 159,
 142,
 126,
 134,
 135,
 136,
 136,
 135,
 130,
 136,
 138,
 119,
 135,
 136,
 130,
 126,
 169,
 134,
 140,
 120,
 128,
 126,
 121,
 124,
 142,
 121,

In [5]:
'''Import experiment 2 data'''

header=('subj_x', 'subj_y', 'subj_yaw')
info={'p_goal':[], 'p_obst':[], 'obst_angle':[], 'obst_dist':[], 'subj_id':[], 'trial_id':[], 'stimuli_onset':[],
     'w_goal': 0.1, 'w_obst': 0.1, 'ps':[]}
exp2 = Data(Hz=30, header=header, info=info)

file_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data\\Cohen_movObst_rawData\\Cohen_movObst_exp2_rawData.csv'))
ii = 0
head = 0
tail = 0
# subj_ps = {}
ps = 0
with open(file_path, 'r') as f:
    trials = [line.split(',') for line in f.read().split('\n')]
    head = 0
    while True:
        i_subj = int(trials[head][1])
        i_trial = int(trials[head][3])
        order = int(trials[head][5])
        for i in range(head+1, len(trials)):
            if trials[i][0] == '':
                tail = i - 1
                break
        subj = tuple([(float(line[5])/100, float(line[6])/100, float(line[7])) for line in trials[head+2:tail+1]])
        goal = tuple([(float(line[1])/100, float(line[2])/100) for line in trials[head+2:tail+1]])
        obst = np.array([[float(line[3])/100, float(line[4])/100] for line in trials[head+2:tail+1]])
        onset = 0         
        for i in range(len(subj)):
            if subj[i][1] >= 1:
                onset = i
                break
        exp2.add_traj(subj)
        # Use speed at -1 second as the trial preferred speed
        speed = traj_speed(np.array(exp2.get_traj(len(exp2.trajs)-1))[:,:2], exp2.Hz)
        if len(speed) > 1 * exp2.Hz:
            ps = speed[-1*exp2.Hz]
        else:
            ps = 0
        # Use the speed at the onset of stimuli as the preferred speed of the trial
#         if i_subj in subj_ps:
#             subj_ps[i_subj].append(speed[onset])
#         else:
#             subj_ps[i_subj] = [speed[onset]]
            
        # After inspection, those trials are no good
        if len(obst) < 30:
            exp2.add_dump(ii, 'recording error')
        if ii == 325 or ii == 778:
            exp2.add_dump(ii, 'abnormal speed')
            
        if obst[0, 1] < 5:
            dist = None
        elif obst[0, 1] < 6:
            dist = 'near'
        elif obst[0, 1] < 7:
            dist = 'far'
        
        if obst[-9][1] - obst[-1][1] > 0.1:
            angle = 140
        elif obst[-9][1] - obst[-1][1] > 0.08:
            angle = 130
        elif obst[-9][1] - obst[-1][1] > 0.06:
            angle = 120
        elif obst[-9][1] - obst[-1][1] > 0.04:
            angle = 110
        elif obst[-9][1] - obst[-1][1] > 0.02:
            angle = 100
        else:
            angle = None
        if obst[0,0] < 0 and angle:
            angle = -angle
        obst = tuple([tuple(p) for p in obst])
        exp2.add_info({'p_goal':goal, 'p_obst':obst, 'obst_angle':angle, 'obst_dist':dist, 'subj_id':i_subj, 
                       'trial_id':i_trial, 'stimuli_onset':onset, 'ps':ps})
        if tail + 3 >= len(trials):
            break
        head = tail + 2
        ii += 1
        
# Setting subject preferred speed to every trial
# for i in range(len(exp2.trajs)):
#     i_subj = exp2.info['subj_id'][i]
#     exp2.add_info({'ps': np.mean(subj_ps[i_subj])})
    
outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data\\Cohen_movObst_exp2_data.pickle'))          
with open(outfile, 'wb') as file:   
    pickle.dump(exp2, file, pickle.HIGHEST_PROTOCOL)  

In [182]:
'''Check imported data'''
%matplotlib qt
vals = []
dump = []
speed = traj_speed(np.array(exp2.get_traj(778))[:,0:2], exp2.Hz)
plt.plot(speed)
for i in range(len(exp2.trajs)):
    if i not in exp2.dump:
        onset = exp2.info['stimuli_onset'][i]
        i_subj = exp2.info['subj_id'][i]
        p_subj = np.array(exp2.trajs[i])
        p_goal = np.array(exp2.info['p_goal'][i])
        p_obst = np.array(exp2.info['p_obst'][i])        
        angle = exp2.info['obst_angle'][i]
        dist = exp2.info['obst_dist'][i]
        
        speed = traj_speed(np.array(exp2.get_traj(i))[:,0:2], exp2.Hz)
#         plt.plot(speed)
#         if i == 325 or i == 778:
#             plt.plot(speed)
#         if sum(speed > 3) > 0:
#             print(i)
#         vals.append(exp2.info['ps'][i])
#         plt.plot(vals)

#         print(onset, p_subj[onset][1])